In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

OUTPUT_DIR = "../data/others/model_output"

folder_paths = [OUTPUT_DIR]

# Create the output directories if they doesn't exist
for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from plot import get_bounding_boxes, create_bboxes_image
import cv2
import numpy as np
from layers import AMF_GD_YOLOv8
import matplotlib.pyplot as plt
from time import time
import random
import torch
import torch.nn as nn
from utils import Folders
from data_processing import ImageData
from tqdm.notebook import tqdm
from typing import Sequence, Dict, Any, Callable, List, Tuple
import os
from torch.utils.data import Dataset, DataLoader
from utils import get_file_base_name
from skimage import io

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Run the YOLOv8 model

In [4]:
from ultralytics.models.yolo import YOLO

# Load a pretrained YOLO model
model = YOLO("yolov8n.pt")
results = model.train(data="coco8.yaml", epochs=3)

# Perform object detection on an image using the model
# results = model("https://ultralytics.com/images/bus.jpg", save=True)

New https://pypi.org/project/ultralytics/8.2.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8 🚀 Python-3.12.3 torch-2.1.2.post304 CUDA:0 (NVIDIA GeForce GTX 1080, 8111MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

train: Scanning /home/alexandre/Documents/tree-segmentation/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /home/alexandre/Documents/tree-segmentation/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        3/3      0.77G       1.34      2.335      1.458         27        640: 100%|██████████| 1/1 [00:00<00:00,  9.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 26.26it/s]

                   all          4         17      0.589       0.89      0.888      0.619



3 epochs completed in 0.001 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.8 🚀 Python-3.12.3 torch-2.1.2.post304 CUDA:0 (NVIDIA GeForce GTX 1080, 8111MiB)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 32.74it/s]


                   all          4         17      0.617      0.886      0.888      0.621
                person          4         10      0.669        0.5      0.521      0.278
                   dog          4          1       0.36          1      0.995      0.597
                 horse          4          2      0.727          1      0.995      0.631
              elephant          4          2      0.605      0.816      0.828      0.332
              umbrella          4          1      0.552          1      0.995      0.995
          potted plant          4          1      0.788          1      0.995      0.895
WARNING ⚠️ ConfusionMatrix plot failure: 
Speed: 0.3ms preprocess, 3.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train2


### Use the AMF GD YOLOv8 model

In [6]:
from ultralytics.engine.trainer import BaseTrainer
from ultralytics.models.yolo.detect.train import DetectionTrainer

class_names = {
    0: "Tree",
    1: "Tree_unsure",
    2: "Tree_disappeared",
    3: "Tree_replaced",
    4: "Tree_new",
}

class_indices = {value: key for key, value in class_names.items()}

model = AMF_GD_YOLOv8(3, 1, device=device, scale="s", class_names=class_names)

results = model.train(data="coco8.yaml", trainer=BaseTrainer, epochs=3)

# rgb_image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.tif"
# chm_image_path = (
#     "../data/CHM_cropped_0p24/122000_484000/122000_484000_1_1_filtered_chm.tif"
# )
# annotations_path = "../data/annotations/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.json"

# gt_bboxes, gt_classes = get_bounding_boxes(annotations_path)

# output_name = "Model_output_test.jpg"
# output_path = os.path.join(OUTPUT_DIR, output_name)

# model.train()
# output = model(rgb_image_path, chm_image_path, image_save_path=output_path)

New https://pypi.org/project/ultralytics/8.2.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8 🚀 Python-3.12.3 torch-2.1.2.post304 CUDA:0 (NVIDIA GeForce GTX 1080, 8111MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

NotImplementedError: This task trainer doesn't support loading cfg files

In [ ]:
# from ultralytics.models.utils.loss import DETRLoss
# from layers import xywh2xyxy

# nc = 4
# loss_func = DETRLoss(nc=nc)

# # output[0][0][0, :, 0] = torch.Tensor([107.3209, 151.2296, 161.4145, 207.6499, 0.9837, 0.5472, 0.3150, 0.5183])

# pred_bboxes = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, :4]
# pred_scores = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, 4:]
# batch = {
#     "cls": torch.Tensor([class_indices[cls] for cls in classes]).to(dtype=torch.int64),
#     "bboxes": xywh2xyxy(torch.Tensor(bboxes)),
#     "gt_groups": [len(classes)],
# }

# prefect_pred_bboxes = batch["bboxes"].unsqueeze(0).unsqueeze(0)
# prefect_pred_scores = (
#     torch.where(
#         batch["cls"].unsqueeze(-1).repeat(1, nc)
#         == torch.arange(nc).unsqueeze(0).repeat(batch["cls"].shape[0], 1),
#         1.0,
#         -1.0,
#     )
#     .unsqueeze(0)
#     .unsqueeze(0)
# )

# loss = loss_func.forward(pred_bboxes, pred_scores, batch)
# print(f"{loss = }")

model.train()
for i in range(100):
    output = model(rgb_image_path, chm_image_path, image_save_path=output_path)
    loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
    print(loss)
    (loss["loss_class"] + loss["loss_bbox"]).backward()

In [ ]:
num_epochs = 100

dataloader = ...
model = AMF_GD_YOLOv8(3, 1, scale="n", class_names=class_names)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lambda i: 1 / np.sqrt(i + 2), last_epoch=-1, verbose=True
)

model.train()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        rgb_image = ...
        chm_image = ...

        output = model(rgb_image, chm_image)
        loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
        # total_loss = loss["loss_class"] + loss["loss_bbox"]
        total_loss = torch.sum(torch.stack(list(loss.values())), dim=0).float()

        optimizer.zero_grad()

        total_loss.backward()

        optimizer.step()

    scheduler.step()
    print(f"Loss class: {loss["loss_class"]:.4f}, Loss bbox: {loss["loss_bbox"]:.4f}")

### How to use data augmentation with Albumentations

In [ ]:
class Pad(A.PadIfNeeded):
    def __init__(
        self,
        added_height: int | None = 64,
        added_width: int | None = 64,
        pad_height_divisor: int | None = None,
        pad_width_divisor: int | None = None,
        position: A.PadIfNeeded.PositionType | str = A.PadIfNeeded.PositionType.CENTER,
        border_mode: int = cv2.BORDER_REFLECT_101,
        value: float | Sequence[float] | None = None,
        mask_value: float | Sequence[float] | None = None,
        always_apply: bool = False,
        p: float = 1,
    ):
        super().__init__(
            0,
            0,
            pad_height_divisor,
            pad_width_divisor,
            position,
            border_mode,
            value,
            mask_value,
            always_apply,
            p,
        )
        self.added_height = added_height
        self.added_width = added_width

    def update_params(self, params: Dict[str, Any], **kwargs: Any) -> Dict[str, Any]:
        params = super().update_params(params, **kwargs)
        rows = params["rows"]
        cols = params["cols"]
        self.min_height = rows + self.added_height
        self.min_width = cols + self.added_width
        return super().update_params(params, **kwargs)

In [4]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

bbox_params = A.BboxParams(
    format="pascal_voc", min_area=0, min_visibility=0.2, label_fields=["class_labels"]
)

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=bbox_params,
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(p=1.0),
    ],
)

image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.tif"
annotations_path = (
    "../data/annotations/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.json"
)
# image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
# annotations_path = "../data/annotations/full/10"

t = time()

image = cv2.imread(image_path)
bboxes, labels = get_bounding_boxes(annotations_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 10

colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

t = time()

for i in range(number_tests):
    transformed_spatial = transform_spatial(
        image=image, bboxes=[bbox.as_list() for bbox in bboxes], class_labels=labels
    )
    transformed_spatial_image = transformed_spatial["image"]
    transformed_bboxes = transformed_spatial["bboxes"]
    transformed_class_labels = transformed_spatial["class_labels"]
    transformed = transform_pixel(image=transformed_spatial_image)
    transformed_image = transformed["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    bboxes_image = create_bboxes_image(
        transformed_image,
        transformed_bboxes,
        labels=transformed_class_labels,
        colors_dict=colors,
        scores=np.random.rand((len(bboxes))),
    )

    output_name = f"Augmentation_test_{i}_bboxes.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, bboxes_image)

print(f"Time elapsed: {round(time() - t, 3)}")

Time elapsed: 0.009
Time elapsed: 1.179


### PyTorch dataset

In [4]:
full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(
    Folders.CROPPED_ANNOTS.value, image_data.base_name
)

rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

means_rgb = torch.zeros(3)
means_chm = 0
count = 0

for file_name in tqdm(os.listdir(annotations_folder_path)):
    annotations_file_path = os.path.join(annotations_folder_path, file_name)
    base_name = get_file_base_name(annotations_file_path)
    rgb_path = os.path.join(rgb_folder_path, f"{base_name}.tif")
    image_rgb = io.imread(rgb_path)
    chm_path = os.path.join(chm_folder_path, f"{base_name}.tif")
    image_chm = io.imread(chm_path)

    means_rgb += np.std(image_rgb, axis=(0, 1))
    image_chm = np.where(image_chm == -9999, 0, image_chm)
    means_chm += np.std(image_chm)
    count += 1

mean_rgb = means_rgb / count
mean_chm = means_chm / count

print(f"{mean_rgb = }")
print(f"{mean_chm = }")

/home/alexandre/miniforge3/envs/tree-segment/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


mean_rgb = tensor([34.3328, 32.1673, 27.6653], dtype=torch.float64)
mean_chm = 4.555709674383501


In [4]:
def normalize_rgb(image_rgb: torch.Tensor) -> torch.Tensor:
    ## On the whole image
    # mean_rgb = torch.tensor([78.152, 88.417, 86.365]).view(-1, 1, 1)
    # std_rgb = torch.tensor([45.819, 42.492, 38.960]).view(-1, 1, 1)
    ## On the labeled parts
    mean_rgb = torch.tensor([77.692, 89.142, 85.816]).view(-1, 1, 1)
    std_rgb = torch.tensor([34.3328, 32.1673, 27.6653]).view(-1, 1, 1)
    return (image_rgb - mean_rgb) / std_rgb


def normalize_chm(image_chm: torch.Tensor) -> torch.Tensor:
    image_chm = torch.where(image_chm == -9999, 0, image_chm)
    ## On the whole image
    # mean_chm = 2.4113
    # std_chm = 5.5642
    ## On the labeled parts
    mean_chm = 3.0424
    std_chm = 4.5557
    return (image_chm - mean_chm) / std_chm


class TreeDataset(Dataset):
    """Tree dataset."""

    def __init__(
        self,
        annotations_folder_path: str,  # TODO: Change to take a list of files instead (to separate train and validation)
        rgb_folder_path: str,
        chm_folder_path: str,
        labels_to_index: Dict[str, int],
        labels_to_color: Dict[str, Tuple[int, int, int]],
        transform_spatial: Callable | None = None,
        transform_pixel: Callable | None = None,
    ) -> None:
        self.annotations_list: List[str] = []
        self.bboxes: Dict[str, List[List[float]]] = {}
        self.labels: Dict[str, List[int]] = {}
        for file_name in os.listdir(annotations_folder_path):
            annotations_file_path = os.path.join(annotations_folder_path, file_name)
            base_name = get_file_base_name(annotations_file_path)
            self.annotations_list.append(base_name)
            bboxes, labels = get_bounding_boxes(annotations_file_path)
            self.bboxes[base_name] = [bbox.as_list() for bbox in bboxes]
            self.labels[base_name] = [labels_to_index[label] for label in labels]

        self.rgb_folder_path = rgb_folder_path
        self.chm_folder_path = chm_folder_path
        self.labels_to_index = labels_to_index
        self.labels_to_str = {value: key for key, value in self.labels_to_index.items()}
        self.labels_to_color = labels_to_color
        self.transform_spatial = transform_spatial
        self.transform_pixel = transform_pixel

    def __len__(self) -> int:
        # return len(self.landmarks_frame)
        return len(self.annotations_list)

    def get_unnormalized(self, idx) -> Dict[str, torch.Tensor]:
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()

        # Read the images
        base_name = self.annotations_list[idx]
        rgb_path = os.path.join(self.rgb_folder_path, f"{base_name}.tif")
        image_rgb = io.imread(rgb_path)
        chm_path = os.path.join(self.chm_folder_path, f"{base_name}.tif")
        image_chm = io.imread(chm_path)

        # Get bboxes and labels
        bboxes = self.bboxes[base_name]
        labels = self.labels[base_name]

        # Apply the spatial transform to the two images, bboxes and labels
        if self.transform_spatial is not None:
            transformed_spatial = transform_spatial(
                image=image_rgb, image_chm=image_chm, bboxes=bboxes, class_labels=labels
            )
            image_rgb = transformed_spatial["image"]
            image_chm = transformed_spatial["image_chm"]
            bboxes = transformed_spatial["bboxes"]
            labels = transformed_spatial["class_labels"]

        # Apply the pixel transform the to RGB image
        if transform_pixel is not None:
            transformed = transform_pixel(image=image_rgb)
            image_rgb = transformed["image"]

        to_tensor = ToTensorV2()

        sample = {
            "image_rgb": to_tensor(image=image_rgb)["image"],
            "image_chm": to_tensor(image=image_chm)["image"],
            "bboxes": torch.tensor(bboxes),
            "labels": torch.tensor(labels),
        }
        return sample

    def __getitem__(self, idx):
        sample = self.get_unnormalized(idx)
        sample["image_rgb"] = normalize_rgb(sample["image_rgb"])
        sample["image_chm"] = normalize_chm(sample["image_chm"])

        return sample

    def get_rgb_image(self, idx) -> np.ndarray:
        base_name = self.annotations_list[idx]
        rgb_path = os.path.join(self.rgb_folder_path, f"{base_name}.tif")
        image_rgb = io.imread(rgb_path)
        return image_rgb


def custom_collate_fn(batch):
    # Initialize lists to hold the extracted components
    rgb_images = []
    chm_images = []
    bboxes = []
    labels = []
    indices = []

    last_idx = 0

    # Iterate through the batch
    for item in batch:
        # Extract the components from the dictionary
        rgb_image = item["image_rgb"]
        chm_image = item["image_chm"]
        bbox = item["bboxes"]
        label = item["labels"]

        # Append the extracted components to the lists
        rgb_images.append(rgb_image)
        chm_images.append(chm_image)
        bboxes.append(bbox)
        labels.append(label)

        next_idx = last_idx + len(bbox)
        indices.append([last_idx, next_idx])
        last_idx = next_idx

    # Convert the lists to tensors and stack them
    rgb_images = torch.stack(rgb_images, dim=0)
    chm_images = torch.stack(chm_images, dim=0)
    # bboxes = torch.stack(
    #     [torch.cat((bbox, -torch.ones(max_len - len(bbox), 4))) for bbox in bboxes],
    #     dim=0,
    # )
    # labels = torch.stack(
    #     [torch.cat((label, -torch.ones(max_len - len(label)))) for label in labels],
    #     dim=0,
    # )
    bboxes = torch.cat(bboxes).to(torch.float32)
    labels = torch.cat(labels)
    indices = torch.tensor(indices)

    batch = {
        "image_rgb": rgb_images,
        "image_chm": chm_images,
        "bboxes": bboxes,
        "labels": labels,
        "indices": indices,
    }

    return batch

In [5]:
from collections import defaultdict


class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0.0, "count": 0, "avg": 0.0})

    def update(self, metric_name: str, val: float, count: int = 1):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += count
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                f"{metric_name}: {metric["avg"]:.{self.float_precision}f}"
                for (metric_name, metric) in self.metrics.items()
            ]
        )


def perfect_preds(
    gt_bboxes: torch.Tensor, gt_classes: torch.Tensor, gt_indices: torch.Tensor
):
    extracted_bboxes = []
    extracted_classes = []
    for idx in gt_indices:
        start_idx, end_idx = idx
        slice_bboxes = gt_bboxes[start_idx:end_idx]
        extracted_bboxes.append(slice_bboxes)
        slice_classes = gt_classes[start_idx:end_idx].long()
        extracted_classes.append(slice_classes)
    scores = [
        20 * nn.functional.one_hot(cls, num_classes=5) - 0.5
        for cls in extracted_classes
    ]
    prefect_preds = [
        torch.cat((bboxes, classes), dim=1).permute((1, 0)).unsqueeze(0)
        for bboxes, classes in zip(extracted_bboxes, scores)
    ]
    perfect_preds = torch.cat(
        [
            torch.cat(
                (
                    pred,
                    torch.zeros(
                        (pred.shape[0], pred.shape[1], 8400 - pred.shape[2])
                    ).to(pred.device),
                ),
                dim=2,
            )
            for pred in prefect_preds
        ]
    )
    # print(f"{perfect_preds.shape = }")
    return perfect_preds


def train(
    train_loader: DataLoader,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    device: torch.device,
):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    for i, data in enumerate(stream, start=1):
        # for name, module in model.named_modules():
        #     print(f"{name}: {next(iter(module.parameters()))[0][0]}")
        #     break
        image_rgb: torch.Tensor = data["image_rgb"]
        image_chm: torch.Tensor = data["image_chm"]
        gt_bboxes: torch.Tensor = data["bboxes"]
        gt_classes: torch.Tensor = data["labels"]
        gt_indices: torch.Tensor = data["indices"]

        image_rgb = image_rgb.to(device, non_blocking=True)
        image_chm = image_chm.to(device, non_blocking=True)
        gt_bboxes = gt_bboxes.to(device, non_blocking=True)
        gt_classes = gt_classes.to(device, non_blocking=True)
        gt_indices = gt_indices.to(device, non_blocking=True)

        output = model(image_rgb, image_chm)
        loss = model.compute_loss(
            output, gt_bboxes, gt_classes, gt_indices, bboxes_format="xyxy"
        )
        # total_loss = loss["loss_class"] + loss["loss_bbox"]
        total_loss = torch.sum(torch.stack(list(loss.values())), dim=0).float()

        batch_size = image_rgb.shape[0]
        metric_monitor.update("Loss", total_loss.item(), batch_size)

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        stream.set_description(f"Epoch: {epoch}. Train.      {metric_monitor}")


def validate(
    val_loader: DataLoader, model: nn.Module, epoch: int, device: torch.device
):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            image_rgb: torch.Tensor = data["image_rgb"]
            image_chm: torch.Tensor = data["image_chm"]
            gt_bboxes: torch.Tensor = data["bboxes"]
            gt_classes: torch.Tensor = data["labels"]
            gt_indices: torch.Tensor = data["indices"]

            image_rgb = image_rgb.to(device, non_blocking=True)
            image_chm = image_chm.to(device, non_blocking=True)
            gt_bboxes = gt_bboxes.to(device, non_blocking=True)
            gt_classes = gt_classes.to(device, non_blocking=True)
            gt_indices = gt_indices.to(device, non_blocking=True)

            output = model(image_rgb, image_chm)
            loss = model.compute_loss(
                output, gt_bboxes, gt_classes, gt_indices, bboxes_format="xyxy"
            )
            # prefect_preds = perfect_preds(gt_bboxes, gt_classes, gt_indices)
            # loss = model.compute_loss(
            #     prefect_preds, gt_bboxes, gt_classes, gt_indices, bboxes_format="xyxy"
            # )
            # total_loss = loss["loss_class"] + loss["loss_bbox"]
            total_loss = torch.sum(torch.stack(list(loss.values())), dim=0).float()

            batch_size = image_rgb.shape[0]
            metric_monitor.update("Loss", total_loss.item(), batch_size)
            stream.set_description(f"Epoch: {epoch}. Validation. {metric_monitor}")


def test_save_output_image(
    model: AMF_GD_YOLOv8,
    val_dataset: TreeDataset,
    epoch: int,
    device: torch.device,
    number_images: int = 1,
):
    saved_images = 0
    number_images = min(len(val_dataset), number_images)
    model.eval()
    while saved_images < number_images:
        sample = val_dataset[saved_images]
        image_rgb: torch.Tensor = sample["image_rgb"].unsqueeze(0)
        image_chm: torch.Tensor = sample["image_chm"].unsqueeze(0)
        image_rgb = image_rgb.to(device, non_blocking=True)
        image_chm = image_chm.to(device, non_blocking=True)
        results = model.predict(image_rgb, image_chm)[2]

        initial_rgb = val_dataset.get_rgb_image(saved_images)
        colors_dict = val_dataset.labels_to_color
        if results.boxes is not None:
            bboxes = results.boxes.xyxy.tolist()
            labels = [results.names[cls.item()] for cls in results.boxes.cls]
            scores = results.boxes.conf.tolist()
        else:
            bboxes = []
            labels = []
            scores = []

        bboxes_image = create_bboxes_image(
            image=initial_rgb,
            bboxes=bboxes,
            labels=labels,
            colors_dict=colors_dict,
            scores=scores,
        )

        output_name = (
            f"Augmentation_test_bboxes_{epoch}_{saved_images}_{len(bboxes)}.tif"
        )
        output_path = os.path.join(OUTPUT_DIR, output_name)
        cv2.imwrite(output_path, bboxes_image)

        saved_images += 1


def train_and_validate(
    model: AMF_GD_YOLOv8,
    train_dataset: TreeDataset,
    val_dataset: TreeDataset,
    lr: float,
    epochs: int,
    batch_size: int,
    num_workers: int,
    device: torch.device,
) -> nn.Module:
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda i: 1 / np.sqrt(i + 2), last_epoch=-1, verbose=True
    )

    test_save_output_image(
        model=model, val_dataset=val_dataset, epoch=0, device=device, number_images=2
    )
    for epoch in range(1, epochs + 1):
        train(train_loader, model, optimizer, epoch, device)
        validate(val_loader, model, epoch, device)
        test_save_output_image(
            model=model,
            val_dataset=val_dataset,
            epoch=epoch,
            device=device,
            number_images=2,
        )
        scheduler.step()
    return model

In [8]:
# import gc

# gc.collect()
# torch.cuda.empty_cache()
# with torch.no_grad():
#     torch.cuda.empty_cache()
# del model

# Model

class_names = {
    0: "Tree",
    1: "Tree_unsure",
    2: "Tree_disappeared",
    3: "Tree_replaced",
    4: "Tree_new",
}

class_colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

class_indices = {value: key for key, value in class_names.items()}

model = AMF_GD_YOLOv8(3, 1, device=device, scale="n", class_names=class_names).to(
    device
)

# Transformations for data augmentation

crop_size = 640
distort_steps = 30
distort_limit = 0.2

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0.2,
        label_fields=["class_labels"],
    ),
    additional_targets={"image_chm": "image"},
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
    ],
)

# Paths to the data

full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(
    Folders.CROPPED_ANNOTS.value, image_data.base_name
)
rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

train_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    labels_to_color=class_colors,
    # transform_spatial=transform_spatial,
    # transform_pixel=transform_pixel,
    transform_spatial=None,
    transform_pixel=None,
)

val_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    labels_to_color=class_colors,
    transform_spatial=None,
    transform_pixel=None,
)

# Training parameters

lr = 1e-2
epochs = 10

batch_size = 4
num_workers = 4

train_and_validate(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    device=device,
)

Adjusting learning rate of group 0 to 7.0711e-03.


Adjusting learning rate of group 0 to 5.7735e-03.


Adjusting learning rate of group 0 to 5.0000e-03.


Adjusting learning rate of group 0 to 4.4721e-03.


Adjusting learning rate of group 0 to 4.0825e-03.


...

Adjusting learning rate of group 0 to 3.3333e-03.


Adjusting learning rate of group 0 to 3.1623e-03.


Adjusting learning rate of group 0 to 3.0151e-03.


Adjusting learning rate of group 0 to 2.8868e-03.


AMF_GD_YOLOv8(
  (amfnet): AMFNet(
    (yolo_backbone_left): YOLOv8Backbone(
      (conv1): Conv0(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (conv2): Conv0(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (c2f2): C2f0(
        (cv1): Conv0(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv0(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum

In [7]:
# def print_device_of_layers(model):
#     for name, module in model.named_modules():
#         try:
#             device = next(iter(module.parameters())).device
#             # if device != torch.device("cuda:0"):
#             #     print(f"{name}: {device}")
#             print(f"{name}: {device}")
#         except StopIteration:
#             pass  # Skip modules without parameters


# print_device_of_layers(model)

for name, module in model.named_modules():
    try:
        print(f"{name}: {next(iter(module.parameters()))[0]}")
    except StopIteration:
        pass  # Skip modules without parameters

: tensor([[[-4.1573, -4.0374, -3.9180],
         [-4.1176, -4.0730, -4.3180],
         [-4.0623, -4.1176, -4.0553]],

        [[ 0.1247,  4.5960,  4.4383],
         [-3.5937,  2.9508,  4.3503],
         [-3.8353, -3.7660, -3.8591]],

        [[-4.1754, -4.2308, -4.3584],
         [-4.4608, -4.5333, -4.4738],
         [-4.1419, -4.4643, -4.5171]]], device='cuda:0', grad_fn=<SelectBackward0>)
amfnet: tensor([[[-4.1573, -4.0374, -3.9180],
         [-4.1176, -4.0730, -4.3180],
         [-4.0623, -4.1176, -4.0553]],

        [[ 0.1247,  4.5960,  4.4383],
         [-3.5937,  2.9508,  4.3503],
         [-3.8353, -3.7660, -3.8591]],

        [[-4.1754, -4.2308, -4.3584],
         [-4.4608, -4.5333, -4.4738],
         [-4.1419, -4.4643, -4.5171]]], device='cuda:0', grad_fn=<SelectBackward0>)
amfnet.yolo_backbone_left: tensor([[[-4.1573, -4.0374, -3.9180],
         [-4.1176, -4.0730, -4.3180],
         [-4.0623, -4.1176, -4.0553]],

        [[ 0.1247,  4.5960,  4.4383],
         [-3.5937,  2.950

In [62]:
rgb_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
chm_path = "../data/chm/8cm/filtered/full/122000_484000.tif"
# rgb_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.tif"
# chm_path = "../data/chm/8cm/filtered/cropped/122000_484000/0_0_640_640.tif"


image_rgb = io.imread(rgb_path)
image_chm = io.imread(chm_path)

# RGB normalization

# rgb_mean = np.zeros(3, dtype=np.float32)
# rgb_std = np.zeros(3, dtype=np.float32)
# for i in range(3):
#     rgb_mean[i] = np.mean(image_rgb[:,:,i])
#     rgb_std[i] = np.std(image_rgb[:,:,i])
# print(f"{rgb_mean = }")
# print(f"{rgb_std = }")

image_rgb = normalize_rgb(image_rgb)

# CHM normalization

# chm_mean = np.mean(image_chm)
# chm_std = np.std(image_chm)
# print(f"{chm_mean = }")
# print(f"{chm_std = }")

image_chm = normalize_chm(image_chm)

### Transformation tests

In [64]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

transform = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
    ],
)

transform_2 = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        # A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(p=1.0),
    ],
)

image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.tif"
# image_path = "../data/flat-design-abstract-outline-background_23-2150616456.jpg"

t = time()

image = io.imread(image_path)
# image = (image - rgb_mean) / rgb_std
# image = image.astype(np.float32)
print(f"{image.dtype = }")

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 1

t = time()

for i in range(number_tests):
    r = random.random()
    random.seed(r)
    transformed_image = transform(image=image)["image"]
    random.seed(r)
    transformed_image_2 = transform_2(image=image)["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    io.imsave(output_path, transformed_image)

    output_name = f"Augmentation_test_{i}_2.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    io.imsave(output_path, transformed_image_2)

print(f"Time elapsed: {round(time() - t, 3)}")

image.dtype = dtype('uint8')
Time elapsed: 0.002
Time elapsed: 0.073
